In [2]:
# import required libraries
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as img
import os
import numpy as np


In [5]:
!ls dataset/

lip_train_set.csv  README.md  val_set
lip_val_set.csv    train_set  vis_annotation.py


In [8]:
# csv file line format:
# ImageID_PersonId.jpg,x1,y1,v1,x2,y2,v2,...x16,y16,v16
# Note: x,y, is the annotation label in (column, row),
#       v stands for visuable
        
# Joint order:
#     1,R_Ankle
#     2,R_Knee
#     3,R_Hip
#     4,L_Hip
#     5,L_Knee
#     6,L_Ankle
#     7,B_Pelvis
#     8,B_Spine
#     9,B_Neck
#     10,B_Head
#     11,R_Wrist
#     12,R_Elbow
#     13,R_Shoulder
#     14,L_Shoulder
#     15,L_Elbow
#     16,L_Wrist
col_names = ['ImageID_PersonId', 'R_Ankle_x', 'R_Ankle_y', 'R_Ankle_v',
            'R_Knee_x', 'R_Knee_y', 'R_Knee_v',
            'R_Hip_x', 'R_Hip_y', 'R_Hip_v',
            'L_Hip_x', 'L_Hip_y', 'L_Hip_v',
            'L_Knee_x', 'L_Knee_y', 'L_Knee_v',
            'L_Ankle_x', 'L_Ankle_y', 'L_Ankle_v',
            'B_Pelvis_x', 'B_Pelvis_y', 'B_Pelvis_v',
            'B_Spine_x', 'B_Spine_y', 'B_Spine_v',
            'B_Neck_x', 'B_Neck_y', 'B_Neck_v',
            'B_Head_x', 'B_Head_y', 'B_Head_v',
            'R_Wrist_x', 'R_Wrist_y', 'R_Wrist_v',
            'R_Elbow_x', 'R_Elbow_y', 'R_Elbow_v',
            'R_Shoulder_x', 'R_Shoulder_y', 'R_Shoulder_v',
            'L_Shoulder_x', 'L_Shoulder_y', 'L_Shoulder_v',
            'L_Elbow_x', 'L_Elbow_y', 'L_Elbow_v',
            'L_Wrist_x', 'L_Wrist_y', 'L_Wrist_v']

In [9]:
# read csv containing image name and corresponding keypoints
train_df = pd.read_csv('dataset/lip_train_set.csv', names=col_names)
train_df.head()

,ImageID_PersonId,R_Ankle_x,R_Ankle_y,R_Ankle_v,R_Knee_x,R_Knee_y,R_Knee_v,R_Hip_x,R_Hip_y,R_Hip_v,...,R_Shoulder_v,L_Shoulder_x,L_Shoulder_y,L_Shoulder_v,L_Elbow_x,L_Elbow_y,L_Elbow_v,L_Wrist_x,L_Wrist_y,L_Wrist_v
0,1000_1234574.jpg,32.0,196.0,0.0,33.0,163.0,0.0,31.0,115.0,0.0,...,0.0,70.0,47.0,0.0,82.0,68.0,0.0,71.0,87.0,0.0
1,100111_470108.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,122.0,140.0,1.0,179.0,174.0,0.0,244.0,143.0,0.0
2,10012_1730659.jpg,82.0,272.0,0.0,71.0,206.0,0.0,67.0,126.0,0.0,...,0.0,16.0,60.0,0.0,12.0,111.0,0.0,4.0,154.0,0.0
3,10014_189863.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,17.0,115.0,0.0,10.0,200.0,0.0,NaN,NaN,NaN
4,100159_1748699.jpg,14.0,270.0,0.0,18.0,205.0,0.0,14.0,133.0,0.0,...,0.0,80.0,61.0,0.0,87.0,113.0,0.0,77.0,154.0,0.0


In [ ]:
from PIL import Image, ImageDraw

def plot_joint(rec, img_folder):
	# img_name = os.path.join(img_folder, 'images',rec[0])
	img_name = os.path.join(img_folder, rec[0])
	print('Image at: ' + img_name)
	
	img = Image.open(img_name)
	draw = ImageDraw.Draw(img)
	r = 5
	bombs = [[0,1],[1,2]
			,[3,4],[4,5]
			,[6,7],[7,8],[8,9]
			,[10,11],[11,12]
			,[13,14],[14,15] ]
	colors = [(255,0,0),(255,0,0),
			  (0,255,0),(0,255,0),
			  (0,0,255),(0,0,255),(0,0,255),
			  (128,128,0),(128,128,0),
			  (128,0,128),(128,0,128)]
	r = 5 
	for b_id in range(len(bombs)):
		b = bombs[b_id]
		color = colors[b_id]
		x1 = rec[ b[0] * 3 + 1]
		y1 = rec[ b[0] * 3 + 2]
		v1 = rec[ b[0] * 3 + 3]

		x2 = rec[ b[1] * 3 + 1]
		y2 = rec[ b[1] * 3 + 2]
		v2 = rec[ b[1] * 3 + 3]
		
		if v1 != 'nan' and v2 != 'nan':
			draw.line((int(x1),int(y1), int(x2),int(y2)), fill = color, width = 5)
		elif v1 != 'nan':
			draw.ellipse((int(x1) - r, int(y1) - r, int(x1) + r, int(y1) + r), fill = color)
		elif v2 != 'nan':
			draw.ellipse((int(x2) - r, int(y2) - r, int(x2) + r, int(y2) + r), fill = color)

	img.show()